In [ ]:
# @title 1️⃣ SETUP: Clone Project & Submodules
import os
import shutil
import subprocess
import torch

# --- CẤU HÌNH ---
REPO_URL = "https://github.com/rishubhpar/Depth-Aware-Editing.git"
TEMP_DIR = "/kaggle/temp/Depth-Aware-Editing"
WORK_LINK = "/kaggle/working/Depth-Aware-Editing"

print("🚀 BẮT ĐẦU KHỞI TẠO (Mode: Stable Diffusion 1.5)...")

# 1. Kiểm tra GPU
if not torch.cuda.is_available():
    print("⚠️ CẢNH BÁO: Chưa bật GPU! Vào Session Options -> Accelerator -> GPU T4 x2.")
else:
    print(f"✅ GPU OK: {torch.cuda.get_device_name(0)}")

def run_cmd(cmd):
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL)

# 2. Dọn dẹp & Clone
if os.path.exists("/kaggle/working"): os.chdir("/kaggle/working")
if os.path.exists(WORK_LINK):
    if os.path.islink(WORK_LINK): os.unlink(WORK_LINK)
    else: shutil.rmtree(WORK_LINK)
if os.path.exists(TEMP_DIR): shutil.rmtree(TEMP_DIR)

print("⬇️ Đang clone Repository chính...")
run_cmd(f"git clone {REPO_URL} {TEMP_DIR}")
os.symlink(TEMP_DIR, WORK_LINK)
os.chdir(TEMP_DIR)

# 3. Clone Submodules Thủ công
print("🏗️ Đang clone các Module con (SAM & GroundingDINO)...")
run_cmd("git clone https://github.com/IDEA-Research/Grounded-Segment-Anything.git src/grounded_sam")
if os.path.exists("src/grounded_sam/GroundingDINO"): shutil.rmtree("src/grounded_sam/GroundingDINO")
run_cmd("git clone https://github.com/IDEA-Research/GroundingDINO.git src/grounded_sam/GroundingDINO")

print("✅ BƯỚC 1 HOÀN TẤT.")

In [ ]:
# @title 2️⃣ INSTALL: Cài đặt Thư viện (FORCE OVERRIDE VERSION)
import os
import subprocess
import sys

# Đảm bảo làm việc tại thư mục temp
os.chdir("/kaggle/temp/Depth-Aware-Editing")

print("📦 Đang cài đặt thư viện (Chế độ Cưỡng bức Version)...")
print("⚠️ Code này sẽ gỡ và cài lại nhiều lần để ép đúng version. Hãy kiên nhẫn!\n")

def run_cmd(cmd, desc=""):
    if desc:
        print(f"    ⬇️ {desc}...")
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# ============================================================
# BƯỚC 1: DỌN DẸP SẠCH SẼ
# ============================================================
print("🧹 Bước 1/6: Gỡ bỏ các thư viện gây xung đột...")
uninstall_pkgs = [
    "numpy", "scipy", "scikit-learn", "matplotlib", "torchmetrics",
    "torch", "torchvision", "torchaudio", "xformers", 
    "pytorch-lightning", "lightning",
    "transformers", "diffusers", "accelerate", "huggingface_hub", "peft",
    "gradio", "opencv-python", "opencv-python-headless", "albumentations",
    "segment-anything", "groundingdino", 
    "tensorflow", "keras", "tensorboard", 
    "jax", "jaxlib"
]
run_cmd(f"pip uninstall -y {' '.join(uninstall_pkgs)}")

# ============================================================
# BƯỚC 2: CÀI PYTORCH & CUDA
# ============================================================
print("🔥 Bước 2/6: Cài đặt PyTorch 2.1...")
run_cmd(
    "pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 "
    "--index-url https://download.pytorch.org/whl/cu121"
)
run_cmd(
    "pip install xformers==0.0.23 post1 --index-url https://download.pytorch.org/whl/cu121"
)

# ============================================================
# BƯỚC 3: CÀI CÁC THƯ VIỆN NỀN TẢNG (Transformers, Accelerate...)
# ============================================================
print("🤖 Bước 3/6: Cài đặt các thư viện nền (Cho phép cài Hub mới tạm thời)...")
# Ở bước này, ta cứ để nó cài bản mới nhất nếu nó muốn, ta sẽ sửa sau
ai_libs = [
    "opencv-python-headless", "albumentations", "imageio", "moviepy",
    "transformers==4.39.3", "accelerate==0.28.0",
    "safetensors", "omegaconf", "einops", "timm",
    "addict", "yapf", "pyyaml", "gradio==3.50.2", "protobuf==3.20.3",
    "scikit-learn", "matplotlib", 
    "pytorch-lightning==2.0.9", "torchmetrics==1.2.1"
]
run_cmd(f"pip install {' '.join(ai_libs)}")

# ============================================================
# BƯỚC 4: CÀI MODULES NGOÀI (SAM & DINO)
# ============================================================
print("🎯 Bước 4/6: Cài đặt SAM & GroundingDINO...")
run_cmd("pip install git+https://github.com/facebookresearch/segment-anything.git")
run_cmd("pip install git+https://github.com/IDEA-Research/GroundingDINO.git")

# ============================================================
# BƯỚC 5: CƯỠNG BỨC CÀI ĐẶT CÁC VERSION QUAN TRỌNG (THE FIX)
# ============================================================
print("🔨 Bước 5/6: Cưỡng bức cài đặt Diffusers, PEFT và Hub cũ...")

# 1. Gỡ bỏ bản Hub mới (0.36.0) mà các thư viện trên vừa lén cài vào
run_cmd("pip uninstall -y huggingface_hub diffusers peft")

# 2. Cài đặt lại với cờ --no-deps (Không cài dependencies phụ thuộc để tránh update ngược lại)
# Đây là bước quan trọng nhất: Ép buộc dùng đúng version này.
install_force = [
    "huggingface_hub<0.25.0",  # Fix lỗi cached_download
    "diffusers==0.27.2",       # Fix lỗi GLIGEN
    "peft==0.8.2"              # Fix lỗi tương thích
]
for pkg in install_force:
    print(f"      👉 Ép cài: {pkg}")
    run_cmd(f"pip install '{pkg}' --force-reinstall --no-deps")

# ============================================================
# BƯỚC 6: CHỐT HẠ VERSION NUMPY & SCIPY
# ============================================================
print("🔨 Bước 6/7: Ép phiên bản NumPy 1.26.4...")
run_cmd("pip uninstall -y jax jaxlib numpy scipy")
run_cmd("pip install 'numpy==1.26.4' 'scipy==1.11.4' --force-reinstall")

# THÊM: Fix numpy.dtype binary incompatibility
print("🔧 Bước 7/7: Recompile packages phụ thuộc numpy...")
run_cmd("pip install --force-reinstall --no-cache-dir scikit-learn matplotlib pillow opencv-python-headless")

# ============================================================
# KIỂM TRA KẾT QUẢ
# ============================================================
print("\n" + "="*60)
print("🔍 KIỂM TRA PHIÊN BẢN (FINAL CHECK):")
print("="*60)

check_cmds = [
    ("NumPy (Target: 1.26.4)", "import numpy; print(numpy.__version__)"),
    ("Diffusers (Target: 0.27.2)", "import diffusers; print(diffusers.__version__)"),
    ("HuggingFace Hub (Target: <0.25)", "import huggingface_hub; print(huggingface_hub.__version__)"),
    ("PEFT (Target: 0.8.2)", "import peft; print(peft.__version__)")
]

for name, cmd in check_cmds:
    result = subprocess.run([sys.executable, "-c", cmd], capture_output=True, text=True)
    ver = result.stdout.strip() if result.returncode == 0 else "ERROR"
    
    # Logic kiểm tra màu
    status = "✅"
    if "ERROR" in ver: 
        status = "❌"
    elif name.startswith("HuggingFace") and ver.startswith("0.3"): # Nếu Hub lại nhảy lên 0.3x
        status = "❌ (Vẫn bị ghi đè!)"
    
    print(f"    {status} {name}: {ver}")

print("="*60)
print("✅ QUÁ TRÌNH CÀI ĐẶT HOÀN TẤT!")

print("\n⚠️  BƯỚC QUYẾT ĐỊNH: RESTART SESSION NGAY.")
print("="*60)

print("    👉 Chọn Menu: Run -> Restart Session")
print("    👉 Sau khi Restart: Chạy tiếp Cell 3 (đã sửa ở trên), rồi Cell 4, 5.")

In [1]:
# @title 3️⃣ PATCH: Vá lỗi TOÀN DIỆN (Bản "Direct Import" - Bỏ qua torch.hub)
import os
import subprocess
import sys
import shutil

WORK_DIR = "/kaggle/temp/Depth-Aware-Editing"
if os.path.exists(WORK_DIR):
    os.chdir(WORK_DIR)
else:
    raise FileNotFoundError(f"Không tìm thấy {WORK_DIR}. Chạy lại Cell 1!")

print("🚑 Đang chạy bản vá lỗi (Chiến thuật: Direct Import - Loại bỏ torch.hub)...")

def run_cmd(cmd):
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# ============================================================
# 1. CÀI THƯ VIỆN & SETUP DINOv2 (QUAN TRỌNG)
# ============================================================
print("    📥 [1/10] Setup môi trường & DINOv2...")
run_cmd("pip install open-clip-torch")

# Clone và copy DINOv2 ra root để Python nhìn thấy trực tiếp
dinov2_repo = os.path.join(os.getcwd(), "dinov2_repo")
if not os.path.exists(dinov2_repo):
    subprocess.run(f"git clone -q https://github.com/facebookresearch/dinov2.git {dinov2_repo}", shell=True)

# Copy folder 'dinov2' (package) ra thư mục gốc
source_dinov2 = os.path.join(dinov2_repo, "dinov2")
target_dinov2 = os.path.join(os.getcwd(), "dinov2")

if os.path.exists(source_dinov2):
    if os.path.exists(target_dinov2):
        shutil.rmtree(target_dinov2)
    shutil.copytree(source_dinov2, target_dinov2)
    print("      ✅ Đã đặt DINOv2 vào vị trí chuẩn.")

# ============================================================
# 2. VÁ FILE modules.py (LOẠI BỎ torch.hub.load)
# ============================================================
print("    🔧 [2/10] Phẫu thuật file modules.py (Bypass torch.hub)...")
modules_file = "ldm/modules/encoders/modules.py"

if os.path.exists(modules_file):
    with open(modules_file, "r") as f:
        code = f.read()

    # Đoạn code mới: Import trực tiếp thay vì dùng torch.hub
    header_patch = """
import sys, os
# Đảm bảo Python tìm thấy dinov2 ở thư mục hiện tại
sys.path.append(os.getcwd()) 
try:
    from dinov2.hub.backbones import dinov2_vitg14
except ImportError:
    print("⚠️ Warning: Import dinov2 failed initially, retrying...")
    from dinov2_repo.dinov2.hub.backbones import dinov2_vitg14
"""
    if "from dinov2.hub.backbones" not in code:
        code = header_patch + "\n" + code

    # Thay thế chuỗi string gọi hàm
    target_str_1 = "torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')"
    target_str_2 = "torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14', source='local', pretrained=False)"
    target_str_3 = f"torch.hub.load('{dinov2_repo}', 'dinov2_vitg14', source='local', pretrained=False)"
    
    replacement = "dinov2_vitg14(pretrained=False)"
    
    code = code.replace(target_str_1, replacement)
    code = code.replace(target_str_2, replacement)
    code = code.replace(target_str_3, replacement)
    
    # Xử lý trường hợp code đã bị sửa bởi cell trước (lambda) hoặc biến thể khác
    if "source='local'" in code and "dinov2_vitg14" in code:
        lines = code.split('\n')
        new_lines = []
        for line in lines:
            if "torch.hub.load" in line and "dinov2" in line:
                indent = line[:len(line) - len(line.lstrip())]
                # Thay thế trực tiếp dòng gọi lệnh cũ bằng lệnh mới
                new_line = line.replace("torch.hub.load('/kaggle/temp/Depth-Aware-Editing/dinov2_repo', 'dinov2_vitg14', source='local', pretrained=False)", replacement)
                
                # Fix cho các biến thể khác nếu replace trên không bắt được
                if "torch.hub.load" in new_line: 
                     new_line = indent + f"dinov2 = {replacement}" # <--- ĐÃ SỬA LỖI Ở ĐÂY (thêm dấu ngoặc kép)
                
                new_lines.append(new_line)
            else:
                new_lines.append(line)
        code = '\n'.join(new_lines)

    # Xóa sạch các dòng import hubconf thừa thãi
    code = code.replace("import hubconf", "# import hubconf")
    
    with open(modules_file, "w") as f:
        f.write(code)
    print("      ✅ Đã thay thế torch.hub.load bằng Direct Import.")

# ============================================================
# 3. FIX CÁC LỖI KHÁC (NHƯ CŨ)
# ============================================================
print("    🔄 [3/10] Fix datasets, PositionNet, CaptionProjection...")
if os.path.exists("datasets") and os.path.isdir("datasets"):
    shutil.move("datasets", "repo_datasets")
run_cmd("find . -name '*.py' -print0 | xargs -0 sed -i 's/from datasets/from repo_datasets/g'")
run_cmd("find . -name '*.py' -print0 | xargs -0 sed -i 's/import datasets/import repo_datasets/g'")

# Fix UNet
target_unet = "src/featglac/model/unet_2d_condition.py"
if os.path.exists(target_unet):
    with open(target_unet, "r") as f: code = f.read()
    if "PositionNet," in code: code = code.replace("PositionNet,", "")
    dummy_pos = "import torch.nn as nn\nclass PositionNet(nn.Module):\n    def __init__(self, *args, **kwargs): super().__init__()\n    def forward(self, x, *args, **kwargs): return x"
    if "class PositionNet" not in code:
        code = code.replace("from diffusers.models.embeddings import (", dummy_pos + "\nfrom diffusers.models.embeddings import (")
    with open(target_unet, "w") as f: f.write(code)

# Fix Transformer
target_trans = "src/featglac/model/transformer_2d.py"
if os.path.exists(target_trans):
    with open(target_trans, "r") as f: code = f.read()
    caption_proj = "import torch.nn as nn\nclass CaptionProjection(nn.Module):\n    def __init__(self, i, h): super().__init__(); self.l1=nn.Linear(i,h); self.a=nn.GELU(); self.l2=nn.Linear(h,h)\n    def forward(self, x): return self.l2(self.a(self.l1(x)))"
    if "class CaptionProjection" not in code:
        code = code.replace("from diffusers.models.embeddings import CaptionProjection, PatchEmbed", caption_proj + "\nfrom diffusers.models.embeddings import PatchEmbed")
        code = code.replace("CaptionProjection,", "")
        if "class CaptionProjection" not in code: code = caption_proj + "\n" + code
    with open(target_trans, "w") as f: f.write(code)

# Fix Config & PL
run_cmd("find . -name '*.py' -print0 | xargs -0 sed -i 's|stabilityai/stable-diffusion-2-1-base|runwayml/stable-diffusion-v1-5|g'")
run_cmd("find . -name '*.yaml' -print0 | xargs -0 sed -i 's|stabilityai/stable-diffusion-2-1-base|runwayml/stable-diffusion-v1-5|g'")
run_cmd("find . -name '*.py' -print0 | xargs -0 sed -i 's/pytorch_lightning.utilities.distributed/pytorch_lightning.utilities.rank_zero/g'")

# ============================================================
# 4. VIẾT LẠI FILE grounded_sam_demo1.py
# ============================================================
print("    📝 [4/10] Viết file grounded_sam_demo1.py...")
demo_code = r'''
import sys, os
sys.path.append(os.path.join(os.getcwd(), 'src', 'grounded_sam')); sys.path.append(os.path.join(os.getcwd(), 'src', 'grounded_sam', 'GroundingDINO'))
import argparse, json, numpy as np, torch, cv2
from PIL import Image; import matplotlib.pyplot as plt
import GroundingDINO.groundingdino.datasets.transforms as T
from GroundingDINO.groundingdino.models import build_model; from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
try: from segment_anything import sam_model_registry, sam_hq_model_registry, SamPredictor
except: from segment_anything import sam_model_registry, SamPredictor; sam_hq_model_registry = None
def load_image(p): i = Image.open(p).convert("RGB"); t = T.Compose([T.RandomResize([800], max_size=1333), T.ToTensor(), T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]); return i, t(i, None)[0]
def load_model(c, ck, b, d): a=SLConfig.fromfile(c); a.device=d; a.bert_base_uncased_path=b; m=build_model(a); m.load_state_dict(clean_state_dict(torch.load(ck, map_location="cpu")["model"]), strict=False); m.eval(); return m
def get_grounding_output(m, i, c, b, t, w=True, d="cpu"):
    c=c.lower().strip(); c=c+"." if not c.endswith(".") else c; m=m.to(d); i=i.to(d)
    with torch.no_grad(): o=m(i[None], captions=[c])
    l=o["pred_logits"].cpu().sigmoid()[0]; bx=o["pred_boxes"].cpu()[0]; f=l.max(dim=1)[0]>b; lf=l[f]; bf=bx[f]; tok=m.tokenizer(c); p=[]
    for lg in lf: ph=get_phrases_from_posmap(lg>t, tok, m.tokenizer); p.append(ph+f"({str(lg.max().item())[:4]})" if w else ph)
    return bf, p
def main(a):
    os.makedirs(a.output_dir, exist_ok=True); ip, i=load_image(a.input_image); m=load_model(a.config, a.grounded_checkpoint, a.bert_base_uncased_path, a.device)
    bf, pp=get_grounding_output(m, i, a.text_prompt, a.box_threshold, a.text_threshold, device=a.device)
    s=(sam_hq_model_registry if a.use_sam_hq else sam_model_registry)[a.sam_version](checkpoint=a.sam_hq_checkpoint if a.use_sam_hq else a.sam_checkpoint).to(a.device)
    pr=SamPredictor(s); ic=cv2.cvtColor(cv2.imread(a.input_image), cv2.COLOR_BGR2RGB); pr.set_image(ic)
    H,W=ip.size[1],ip.size[0]
    for k in range(bf.size(0)): bf[k]=bf[k]*torch.Tensor([W,H,W,H]); bf[k][:2]-=bf[k][2:]/2; bf[k][2:]+=bf[k][:2]
    tb=pr.transform.apply_boxes_torch(bf, ic.shape[:2]).to(a.device); ma,_,_=pr.predict_torch(None, None, boxes=tb, multimask_output=False)
    plt.figure(figsize=(10,10)); plt.imshow(ic)
    for mk in ma: c=np.concatenate([np.random.random(3), np.array([0.6])]); h,w=mk.shape[-2:]; plt.gca().imshow(mk.cpu().numpy().reshape(h,w,1)*c.reshape(1,1,-1))
    for bx,lb in zip(bf,pp): x,y,w,h=bx[0],bx[1],bx[2]-bx[0],bx[3]-bx[1]; plt.gca().add_patch(plt.Rectangle((x,y),w,h,edgecolor='green',facecolor='none',lw=2)); plt.gca().text(x,y,lb)
    plt.axis('off'); plt.savefig(os.path.join(a.output_dir, "grounded_sam_output.jpg"), bbox_inches="tight", pad_inches=0); plt.close()
if __name__=="__main__": pass 
'''
with open("src/grounded_sam/grounded_sam_demo1.py", "w") as f: f.write(demo_code)

print("\n" + "="*60)
print("✅ ĐÃ VÁ XONG (Direct Import Strategy)!")
print("👉 Đây là biện pháp mạnh nhất. Hãy chạy lại Cell 4.")
print("="*60)

🚑 Đang chạy bản vá lỗi (Chiến thuật: Direct Import - Loại bỏ torch.hub)...
    📥 [1/10] Setup môi trường & DINOv2...
      ✅ Đã đặt DINOv2 vào vị trí chuẩn.
    🔧 [2/10] Phẫu thuật file modules.py (Bypass torch.hub)...
      ✅ Đã thay thế torch.hub.load bằng Direct Import.
    🔄 [3/10] Fix datasets, PositionNet, CaptionProjection...
    📝 [4/10] Viết file grounded_sam_demo1.py...

✅ ĐÃ VÁ XONG (Direct Import Strategy)!
👉 Đây là biện pháp mạnh nhất. Hãy chạy lại Cell 4.


In [2]:
# @title 4️⃣ WEIGHTS: Chuẩn bị Model
import os

os.chdir("/kaggle/temp/Depth-Aware-Editing")
weights_dir = "weights"
os.makedirs(weights_dir, exist_ok=True)

print("🏋️‍♂️ Đang chuẩn bị Model...")

models = {
    "depth_anything_metric_depth_indoor.pt": ("depth_anything_metric_depth_indoor", "https://huggingface.co/LiheYoung/Depth-Anything/resolve/main/checkpoints_metric_depth/depth_anything_metric_depth_indoor.pt"),
    "depth_anything_vitl14.pth": ("depth_anything_vitl14", "https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth"),
    "dinov2_vitg14_pretrain.pth": ("dinov2_vitg14", "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitg14/dinov2_vitg14_pretrain.pth"),
    "epoch=1-step=8687.ckpt": ("8687", "https://huggingface.co/spaces/xichenhku/AnyDoor/resolve/main/epoch%3D1-step%3D8687.ckpt"),
    "sd-v1-5-inpainting.ckpt": ("inpainting", "https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt")
}

for target, (key, url) in models.items():
    dest = os.path.join(weights_dir, target)
    found = False
    for root, _, files in os.walk("/kaggle/input"):
        for f in files:
            if key in f:
                if os.path.exists(dest) or os.path.islink(dest): os.remove(dest)
                os.symlink(os.path.join(root, f), dest)
                print(f"🔗 Linked: {target}")
                found = True
                break
        if found: break
    
    if not found:
        if not os.path.exists(dest) or os.path.getsize(dest) < 1024*1024:
            print(f"⬇️ Downloading: {target}...")
            os.system(f"wget -q -O '{dest}' '{url}'")
        else:
            print(f"✅ Đã có sẵn: {target}")

print("✅ BƯỚC 4 HOÀN TẤT.")

🏋️‍♂️ Đang chuẩn bị Model...
🔗 Linked: depth_anything_metric_depth_indoor.pt
🔗 Linked: depth_anything_vitl14.pth
🔗 Linked: dinov2_vitg14_pretrain.pth
⬇️ Downloading: epoch=1-step=8687.ckpt...
⬇️ Downloading: sd-v1-5-inpainting.ckpt...
✅ BƯỚC 4 HOÀN TẤT.


In [3]:
# @title 5️⃣ SETUP MODULE 1: Segmentation (FIX: Size Mismatch & NumPy Types)
import os
import sys

# 1. Đảm bảo đường dẫn
if os.path.exists("/kaggle/temp/Depth-Aware-Editing"):
    os.chdir("/kaggle/temp/Depth-Aware-Editing")

print("🚀 Đang thiết lập Module 1 & Vá lỗi kích thước ảnh...")

# ------------------------------------------------------------------------------
# BƯỚC 1: TẠO HUBCONF
# ------------------------------------------------------------------------------
hubconf_code = """
import torch
import os
def dinov2_vitg14(pretrained=False, **kwargs):
    local_repo = os.path.abspath("dinov2_repo")
    if os.path.exists(local_repo):
        try: return torch.hub.load(local_repo, 'dinov2_vitg14', source='local', pretrained=False, **kwargs)
        except: pass
    return torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14', **kwargs)
"""
with open("hubconf.py", "w") as f: f.write(hubconf_code)

# ------------------------------------------------------------------------------
# BƯỚC 2: VÁ FILE MODULES
# ------------------------------------------------------------------------------
target_file = "ldm/modules/encoders/modules.py"
if os.path.exists(target_file):
    with open(target_file, "r") as f: code = f.read()
    if "import hubconf" not in code:
        lines = code.split('\n')
        for i, line in enumerate(lines):
            if line.strip().startswith(("import", "from")):
                lines.insert(i, "import os, sys; sys.path.append(os.getcwd()); import hubconf")
                break
        code = '\n'.join(lines)
    if "torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')" in code:
        code = code.replace("torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')", "hubconf.dinov2_vitg14()")
    with open(target_file, "w") as f: f.write(code)

# ------------------------------------------------------------------------------
# BƯỚC 3: VÁ LỖI NUMPY INT64 (Fix lỗi nội tại của thư viện Depth)
# ------------------------------------------------------------------------------
da_file = "src/Depth-Anything/metric_depth/zoedepth/models/base_models/depth_anything.py"
if os.path.exists(da_file):
    with open(da_file, "r") as f: da_code = f.read()
    target_str = "return nn.functional.interpolate(x, (height, width), mode='bilinear', align_corners=True)"
    replace_str = "return nn.functional.interpolate(x, (int(height), int(width)), mode='bilinear', align_corners=True)"
    if target_str in da_code:
        da_code = da_code.replace(target_str, replace_str)
        with open(da_file, "w") as f: f.write(da_code)
    elif "(height, width)" in da_code: # Fallback
        da_code = da_code.replace("(height, width)", "(int(height), int(width))")
        with open(da_file, "w") as f: f.write(da_code)

# ------------------------------------------------------------------------------
# BƯỚC 4: TẠO FILE RUN GRADIO (Đã thêm lệnh resize đồng bộ)
# ------------------------------------------------------------------------------
gradio_code = r'''
import os
import sys
import torch
import numpy as np
import cv2
from PIL import Image
import gradio as gr

# Setup path
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "grounded_sam"))

# Import functions
from utils.mpi.preprocess import get_depth_and_sam_mask
from utils.mpi.mpi import get_mpi_rgb_and_alpha

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Module 1 Environment: {device}")

def run_segmentation(input_image, depth_threshold):
    if input_image is None:
        raise gr.Error("Vui lòng upload ảnh!")
    
    # 1. Resize ảnh đầu vào (Input)
    w, h = input_image.size
    max_s = 1024
    if max(w, h) > max_s:
        scale = max_s / max(w, h)
        input_image = input_image.resize((int(w * scale), int(h * scale)), Image.LANCZOS)
    
    print("running get_depth_and_sam_mask...")
    # Lấy depth map từ model (Model có thể trả về kích thước hơi khác)
    depth_img, sam_mask_img = get_depth_and_sam_mask(input_image, False)
    
    # --- SỬA LỖI TẠI ĐÂY (QUAN TRỌNG) ---
    # Ép buộc depth_img phải có kích thước Y HỆT input_image
    if depth_img.size != input_image.size:
        print(f"⚠️ Resizing depth from {depth_img.size} to match input {input_image.size}")
        depth_img = depth_img.resize(input_image.size, Image.NEAREST)
    # ------------------------------------

    rgb_np = np.array(input_image)
    depth_np = np.array(depth_img)
    
    # Kiểm tra lần cuối
    if rgb_np.shape[:2] != depth_np.shape[:2]:
        raise ValueError(f"Shape mismatch: RGB {rgb_np.shape} vs Depth {depth_np.shape}")

    # Tách lớp
    thresholds = [(0, depth_threshold), (depth_threshold, 255)]
    print(f"running get_mpi_rgb_and_alpha with threshold {depth_threshold}...")
    
    try:
        mpi_rgb, mpi_alpha = get_mpi_rgb_and_alpha(rgb_np, depth_np, thresholds)
    except cv2.error as e:
        print("OpenCV Error handled:", e)
        # Fallback thủ công nếu OpenCV vẫn lỗi (hiếm khi xảy ra nếu đã resize ở trên)
        mask = (depth_np > depth_threshold).astype(np.uint8) * 255
        mpi_rgb = [cv2.bitwise_and(rgb_np, rgb_np, mask=255-mask), cv2.bitwise_and(rgb_np, rgb_np, mask=mask)]
        mpi_alpha = [(255-mask)/255.0, mask/255.0]

    foreground = mpi_rgb[0]
    background = mpi_rgb[1]
    fg_mask = mpi_alpha[0] * 255
    
    return [
        depth_img,
        sam_mask_img,
        Image.fromarray(foreground.astype(np.uint8)),
        Image.fromarray(background.astype(np.uint8)),
        Image.fromarray(fg_mask.astype(np.uint8))
    ]

with gr.Blocks() as demo:
    gr.Markdown("# 🔍 Module 1: Segmentation & MPI Splitting")
    
    with gr.Row():
        with gr.Column(scale=1):
            input_img = gr.Image(label="Input Image", type="pil", height=400)
            d_thresh = gr.Slider(0, 255, value=50, step=1, label="Depth Threshold")
            btn = gr.Button("✂️ Tách Lớp", variant="primary")
        
        with gr.Column(scale=2):
            gallery = gr.Gallery(label="Kết quả", columns=3, height=600, object_fit="contain")

    btn.click(fn=run_segmentation, inputs=[input_img, d_thresh], outputs=[gallery])

if __name__ == "__main__":
    demo.queue().launch(share=True, debug=True, allowed_paths=["/kaggle/temp"])
'''

with open("gradio_segmentation.py", "w") as f: f.write(gradio_code)
print("✅ Đã sửa lỗi kích thước ảnh (Size Mismatch Fixed).")

🚀 Đang thiết lập Module 1 & Vá lỗi kích thước ảnh...
✅ Đã sửa lỗi kích thước ảnh (Size Mismatch Fixed).


In [4]:
# ------------------------------------------------------------------------------
# BƯỚC 4: TẠO FILE RUN GRADIO (CÓ TÍNH NĂNG LƯU FILE + RAW DEPTH CHO MODULE 2,4)
# ------------------------------------------------------------------------------
gradio_code = r'''
import os
import sys
import torch
import numpy as np
import cv2
from PIL import Image
import gradio as gr
import matplotlib
matplotlib.use('Agg') # Backend không cần màn hình
import matplotlib.pyplot as plt
import io
import traceback

# Setup path
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "grounded_sam"))

from utils.mpi.preprocess import get_depth_and_sam_mask
from utils.mpi.mpi import get_mpi_rgb_and_alpha

# Tạo thư mục output nếu chưa có
OUTPUT_DIR = "outputs/module1"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📂 Kết quả sẽ được lưu tại: {os.path.abspath(OUTPUT_DIR)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- LOAD ZOEDEPTH (KITTI) ---
print("⏳ Đang load model ZoeDepth (ZoeD_K)...")
zoe_model = None
try:
    zoe_model = torch.hub.load("isl-org/ZoeDepth", "ZoeD_K", pretrained=True).to(device).eval()
    print("✅ Load ZoeDepth KITTI thành công!")
except Exception as e:
    print(f"⚠️ Lỗi load model (sẽ dùng fallback): {e}")

def ensure_pil(img):
    if isinstance(img, tuple) or isinstance(img, list): img = img[0]
    if isinstance(img, np.ndarray):
        if img.dtype != np.uint8:
            img = (img - img.min()) / (img.max() - img.min() + 1e-5) * 255
            img = img.astype(np.uint8)
        img = Image.fromarray(img)
    if torch.is_tensor(img):
        img = img.cpu().detach().numpy()
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img)
    return img

def colorize_depth_map(depth_np, is_metric=True):
    plt.figure(figsize=(10, 8))
    if isinstance(depth_np, torch.Tensor): depth_np = depth_np.squeeze().cpu().numpy()
    
    min_val, max_val = np.min(depth_np), np.max(depth_np)
    plt.imshow(depth_np, cmap='jet')
    
    cbar = plt.colorbar(fraction=0.046, pad=0.04)
    if is_metric:
        cbar.set_label('Khoảng cách thực tế (Mét)', rotation=270, labelpad=20)
        plt.title(f"ZoeDepth KITTI (Min: {min_val:.1f}m - Max: {max_val:.1f}m)")
    else:
        cbar.set_label('Giá trị điểm ảnh (Relative)', rotation=270, labelpad=20)
        plt.title(f"Relative Depth (Colorized)")

    plt.axis('off')
    plt.tight_layout()
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0.1)
    buf.seek(0)
    img_color = Image.open(buf).convert("RGB")
    plt.close()
    return img_color

# --- CẬP NHẬT: THÊM THAM SỐ background_img ---
def save_outputs(input_img, mask_img, depth_vis_img, foreground_img, background_img, raw_depth_np):
    """Lưu các file cần thiết cho Module 2 và Module 4"""
    try:
        # 1. Lưu ảnh (Visual)
        input_img.save(os.path.join(OUTPUT_DIR, "input.png"))
        mask_img.save(os.path.join(OUTPUT_DIR, "mask.png"))
        depth_vis_img.save(os.path.join(OUTPUT_DIR, "depth_vis.png"))
        foreground_img.save(os.path.join(OUTPUT_DIR, "foreground.png"))
        
        # --- LƯU BACKGROUND ---
        background_img.save(os.path.join(OUTPUT_DIR, "background.png"))
        
        # 2. LƯU QUAN TRỌNG: RAW DEPTH (NUMPY)
        # File này cần cho Module 4 tính toán 3D
        np.save(os.path.join(OUTPUT_DIR, "depth.npy"), raw_depth_np)
        
        print(f"✅ Đã lưu kết quả (bao gồm background.png và depth.npy) vào {OUTPUT_DIR}")
    except Exception as e:
        print(f"❌ Lỗi khi lưu file: {e}")
        import traceback
        traceback.print_exc()

def run_segmentation(input_image, depth_threshold):
    if input_image is None: raise gr.Error("Chưa upload ảnh!")
    
    # Resize về chuẩn Model (tránh quá to hoặc lẻ)
    w, h = input_image.size
    max_s = 1024
    if max(w, h) > max_s:
        scale = max_s / max(w, h)
        input_image = input_image.resize((int(w * scale), int(h * scale)), Image.LANCZOS)
    
    w, h = input_image.size # Cập nhật lại size
    
    # Biến lưu trữ depth thô (để lưu file .npy)
    final_raw_depth = None 
    depth_metric_img = None
    
    # 1. Metric Depth (ZoeDepth)
    if zoe_model is not None:
        try:
            with torch.no_grad():
                depth_tensor = zoe_model.infer_pil(input_image)
            if isinstance(depth_tensor, torch.Tensor): raw_metric = depth_tensor.cpu().numpy()
            else: raw_metric = depth_tensor
            
            # Resize depth về đúng size ảnh input (nếu Zoe trả về size khác)
            if raw_metric.shape[-2:] != (h, w):
                 raw_metric = cv2.resize(raw_metric, (w, h), interpolation=cv2.INTER_LINEAR)
            
            final_raw_depth = raw_metric # Lưu lại depth thô này
            depth_metric_img = colorize_depth_map(raw_metric, is_metric=True)
        except Exception as e:
            print(f"Zoe Error: {e}")
            depth_metric_img = None

    # 2. Segmentation (SAM / Pretrained)
    print("🚀 Running Segmentation...")
    raw_results = get_depth_and_sam_mask(input_image, False)
    depth_raw_rel = raw_results[0]
    sam_mask_raw = raw_results[1]
    
    depth_img_rel = ensure_pil(depth_raw_rel)
    sam_mask_img = ensure_pil(sam_mask_raw)
    
    # Resize mask về đúng size ảnh input
    if sam_mask_img.size != input_image.size:
        sam_mask_img = sam_mask_img.resize(input_image.size, Image.NEAREST)

    # Fallback Depth: Nếu Zoe lỗi, dùng depth tương đối
    if final_raw_depth is None:
        final_raw_depth = np.array(depth_img_rel).astype(np.float32) / 255.0 # Chuẩn hóa về 0-1
        if depth_metric_img is None:
            depth_metric_img = colorize_depth_map(np.array(depth_img_rel), is_metric=False)

    # 3. MPI / Cutout (Tách nền dựa trên ngưỡng)
    rgb_np = np.array(input_image)
    depth_np_rel = np.array(depth_img_rel)
    
    # Convert mask SAM sang binary (0-255)
    sam_mask_np = np.array(sam_mask_img)
    if len(sam_mask_np.shape) == 3: sam_mask_np = sam_mask_np[:,:,0]
    
    # Tạo ảnh Foreground (Chỉ giữ lại phần trong mask)
    fg_rgba = np.zeros((h, w, 4), dtype=np.uint8)
    fg_rgba[:, :, :3] = rgb_np
    fg_rgba[:, :, 3] = sam_mask_np # Alpha channel
    
    fg_img = Image.fromarray(fg_rgba)
    
    # Tạo ảnh Background (Phần còn lại)
    bg_rgba = np.zeros((h, w, 4), dtype=np.uint8)
    bg_rgba[:, :, :3] = rgb_np
    bg_rgba[:, :, 3] = 255 - sam_mask_np
    bg_img = Image.fromarray(bg_rgba)
    
    alpha_img = Image.fromarray(sam_mask_np)

    # --- LƯU OUTPUT QUAN TRỌNG (ĐÃ THÊM BG_IMG) ---
    save_outputs(input_image, sam_mask_img, depth_metric_img, fg_img, bg_img, final_raw_depth)
    # -------------------------------------------

    return [depth_metric_img, sam_mask_img, fg_img, bg_img, alpha_img]

with gr.Blocks() as demo:
    gr.Markdown("# 🔍 Module 1: Outdoor Segmentation & ZoeDepth (Fixed Auto-Save)")
    gr.Markdown("Kết quả (bao gồm depth map thô) sẽ tự động lưu vào `outputs/module1`.")
    
    with gr.Row():
        with gr.Column(scale=1):
            input_img = gr.Image(label="Input Image", type="pil", height=400)
            d_thresh = gr.Slider(0, 255, value=100, step=1, label="Ngưỡng tách lớp (Không dùng nếu có SAM)")
            btn = gr.Button("🚀 Xử lý & Lưu", variant="primary")
        
        with gr.Column(scale=2):
            gallery = gr.Gallery(label="Kết quả (Depth, Mask, FG, BG, Alpha)", columns=3, height=600, object_fit="contain")

    btn.click(fn=run_segmentation, inputs=[input_img, d_thresh], outputs=[gallery])

if __name__ == "__main__":
    demo.queue().launch(share=True, debug=True, allowed_paths=["/kaggle/temp"])
'''

with open("gradio_segmentation.py", "w") as f: f.write(gradio_code)
print("✅ Đã cập nhật Gradio: Đã sửa lỗi lưu thiếu background.png.")

✅ Đã cập nhật Gradio: Đã sửa lỗi lưu thiếu background.png.


In [5]:
# @title 6️⃣ RUN MODULE 1 (WEIGHTS FIX): Tải Weights & Chạy 10 ảnh
import os
import sys
import gc
import glob
import shutil
import ctypes
import logging
import numpy as np
import torch
import subprocess
from PIL import Image

# Tự động cài pyspark
try:
    from pyspark.sql import SparkSession
except ImportError:
    os.system("pip install pyspark")
    from pyspark.sql import SparkSession

# ==============================================================================
# 🛠️ BƯỚC 1: TỰ ĐỘNG TẢI WEIGHTS (QUAN TRỌNG)
# ==============================================================================
PROJECT_DIR = "/kaggle/working/Depth-Aware-Editing"
WEIGHTS_DIR = os.path.join(PROJECT_DIR, "weights")

print(f"🔧 Đang kiểm tra weights tại: {WEIGHTS_DIR}")

# Kiểm tra xem file bị thiếu có tồn tại không
MISSING_FILE = "depth_anything_metric_depth_indoor.pt"
if not os.path.exists(os.path.join(WEIGHTS_DIR, MISSING_FILE)):
    print(f"⚠️ Phát hiện thiếu file: {MISSING_FILE}")
    print("🔄 Đang chạy script 'download_weights.sh'...")
    
    # Chạy lệnh bash để tải
    try:
        # Cấp quyền thực thi
        subprocess.run(["chmod", "+x", os.path.join(PROJECT_DIR, "download_weights.sh")], check=True)
        # Chạy script trong thư mục dự án
        subprocess.run(["bash", "download_weights.sh"], cwd=PROJECT_DIR, check=True)
        print("✅ Đã tải xong weights!")
    except Exception as e:
        print(f"❌ Lỗi khi tải weights: {e}")
        # Fallback: Tự tải thủ công nếu script lỗi (Link dự phòng)
        if not os.path.exists(WEIGHTS_DIR): os.makedirs(WEIGHTS_DIR)
        print("🌍 Đang thử tải thủ công từ HuggingFace...")
        url = "https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_metric_depth_indoor.pt"
        os.system(f"wget {url} -P {WEIGHTS_DIR}")

# ==============================================================================
# 1. CẤU HÌNH
# ==============================================================================
INPUT_FOLDER = "/kaggle/input/kitti-3d-object-detection-dataset"
OUTPUT_BASE_DIR = "/kaggle/working/Depth-Aware-Editing/spark_output/module1"
TEST_LIMIT = 50 
SPARK_MASTER = "local[1]" 

def clean_memory():
    cleanup_vars = ['model', 'zoe_model', 'pipe', 'diff_handles', 'depth_anything']
    for var in cleanup_vars:
        if var in globals(): del globals()[var]
    gc.collect()
    try: ctypes.CDLL("libc.so.6").malloc_trim(0)
    except: pass
    if torch.cuda.is_available(): torch.cuda.empty_cache()

# ==============================================================================
# 2. LOGIC WORKER (CÓ FIX CWD & PATH)
# ==============================================================================
def spark_worker_logic(iterator):
    import os
    import sys
    import torch
    import cv2
    import numpy as np
    from PIL import Image
    import logging

    # --- 🔥 FIX 1: CHUYỂN CWD VỀ ĐÚNG CHỖ 🔥 ---
    # Code của bạn dùng đường dẫn tương đối "./weights/...", 
    # nên ta phải bắt Worker đứng đúng tại thư mục dự án.
    os.chdir("/kaggle/working/Depth-Aware-Editing")
    # -------------------------------------------

    # --- 🔥 FIX 2: NẠP PATH (Như cũ) ---
    base_dir = os.getcwd() # Lúc này là .../Depth-Aware-Editing
    src_dir = os.path.join(base_dir, "src")
    
    paths_to_inject = [
        os.path.join(src_dir, "ZoeDepth"),
        os.path.join(src_dir, "grounded_sam"),
        src_dir,
        base_dir
    ]
    for p in paths_to_inject:
        if os.path.exists(p) and p not in sys.path:
            sys.path.insert(0, p)
    # -----------------------------------

    # Tắt log
    try:
        from transformers import logging as trans_logging
        trans_logging.set_verbosity_error()
        logging.getLogger("transformers").setLevel(logging.ERROR)
    except: pass

    # Import
    try:
        from utils.mpi.preprocess import get_depth_and_sam_mask
    except ImportError as e:
        yield ("IMPORT_ERROR", str(e), f"CWD: {os.getcwd()}")
        return

    # Load Model ZoeDepth (Nếu code trong preprocess không tự load)
    # Lưu ý: preprocess.py của bạn dường như tự gọi get_depth_map -> tự load model
    # nên ta không cần load thủ công ở đây nữa để tiết kiệm VRAM.
    
    # Loop xử lý
    for row in iterator:
        image_path = row.image_path
        img_id = os.path.basename(image_path).split('.')[0]
        
        try:
            # 1. Đọc ảnh
            input_image = Image.open(image_path).convert("RGB")
            w, h = input_image.size
            
            # Resize
            max_s = 1024
            if max(w, h) > max_s:
                scale = max_s / max(w, h)
                input_image = input_image.resize((int(w * scale), int(h * scale)), Image.LANCZOS)
                w, h = input_image.size
            
            # 2. CHẠY CODE GỐC (Đã fix weights)
            # Hàm này sẽ tự load model từ ./weights/depth_anything_...
            depth_img_rel, sam_mask_img = get_depth_and_sam_mask(input_image, False)
            
            # 3. Chuẩn hóa Depth (Nếu hàm trả về PIL Image)
            final_depth = np.array(depth_img_rel).astype(np.float32) / 255.0
            
            # 4. Lưu kết quả
            save_dir = os.path.join(OUTPUT_BASE_DIR, img_id)
            os.makedirs(save_dir, exist_ok=True)
            
            # Mask
            sam_mask_img = sam_mask_img.resize((w, h), Image.NEAREST)
            sam_mask_np = np.array(sam_mask_img)
            if len(sam_mask_np.shape) == 3: sam_mask_np = sam_mask_np[:,:,0]
            
            # RGBA
            rgb_np = np.array(input_image)
            fg_rgba = np.zeros((h, w, 4), dtype=np.uint8)
            fg_rgba[:, :, :3] = rgb_np
            fg_rgba[:, :, 3] = sam_mask_np
            
            bg_rgba = np.zeros((h, w, 4), dtype=np.uint8)
            bg_rgba[:, :, :3] = rgb_np
            bg_rgba[:, :, 3] = 255 - sam_mask_np

            Image.fromarray(fg_rgba).save(os.path.join(save_dir, "foreground.png"))
            Image.fromarray(bg_rgba).save(os.path.join(save_dir, "background.png"))
            Image.fromarray(sam_mask_np).save(os.path.join(save_dir, "mask.png"))
            input_image.save(os.path.join(save_dir, "input.png"))
            np.save(os.path.join(save_dir, "depth.npy"), final_depth)
            
            yield (img_id, "SUCCESS", save_dir)
            
        except Exception as e:
            yield (img_id, f"ERROR: {str(e)}", "")

# ==============================================================================
# 3. SPARK DRIVER
# ==============================================================================
def main_spark_job():
    clean_memory()
    print(f"🚀 Khởi tạo Spark Session ({SPARK_MASTER})...")
    spark = SparkSession.builder \
        .appName("Module1_WeightsFix") \
        .config("spark.driver.memory", "4g") \
        .master(SPARK_MASTER) \
        .getOrCreate()
    
    # Quét ảnh
    image_files = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        image_files.extend(glob.glob(os.path.join(INPUT_FOLDER, "**", ext), recursive=True))
    
    if len(image_files) > TEST_LIMIT:
        print(f"⚠️ Test {TEST_LIMIT} ảnh.")
        image_files = image_files[:TEST_LIMIT]
        
    if not image_files:
        print("❌ Không tìm thấy ảnh!")
        spark.stop()
        return

    df = spark.createDataFrame([(f,) for f in image_files], ["image_path"])
    df_partitioned = df.repartition(1)
    
    print(f"🔄 Đang xử lý {len(image_files)} ảnh...")
    results_rdd = df_partitioned.rdd.mapPartitions(spark_worker_logic)
    final_results = results_rdd.collect()
    
    print("\n✅ KẾT QUẢ XỬ LÝ:")
    success_count = 0
    for img_id, status, note in final_results:
        if status == "SUCCESS": 
            success_count += 1
            print(f"{img_id[:15]:<15} | {status:<10} | OK")
        else:
            print(f"{img_id[:15]:<15} | {status:<10} | {note[:40]}...")
            
    print(f"🎉 Hoàn thành: {success_count}/{len(final_results)}")
    spark.stop()

if __name__ == "__main__":
    if os.path.exists(OUTPUT_BASE_DIR): shutil.rmtree(OUTPUT_BASE_DIR)
    os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)
    main_spark_job()

🔧 Đang kiểm tra weights tại: /kaggle/working/Depth-Aware-Editing/weights
🚀 Khởi tạo Spark Session (local[1])...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/25 03:28:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


⚠️ Test 50 ảnh.
🔄 Đang xử lý 50 ảnh...


xFormers not available                                              (0 + 1) / 1]
xFormers not available
Params passed to Resize transform:
	width:  518
	height:  392
	resize_target:  True
	keep_aspect_ratio:  False
	ensure_multiple_of:  14
	resize_method:  minimal
Using pretrained resource local::./weights/depth_anything_metric_depth_indoor.pt
Loaded successfully
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



✅ KẾT QUẢ XỬ LÝ:
005084          | SUCCESS    | OK
003065          | SUCCESS    | OK
003741          | SUCCESS    | OK
004725          | SUCCESS    | OK
001183          | SUCCESS    | OK
005660          | SUCCESS    | OK
007355          | SUCCESS    | OK
003244          | SUCCESS    | OK
002594          | SUCCESS    | OK
001703          | SUCCESS    | OK
003380          | SUCCESS    | OK
007190          | SUCCESS    | OK
004566          | SUCCESS    | OK
002584          | SUCCESS    | OK
004035          | SUCCESS    | OK
001840          | SUCCESS    | OK
005647          | SUCCESS    | OK
004924          | SUCCESS    | OK
005000          | SUCCESS    | OK
007456          | SUCCESS    | OK
006796          | SUCCESS    | OK
000419          | SUCCESS    | OK
002712          | SUCCESS    | OK
006386          | SUCCESS    | OK
000920          | SUCCESS    | OK
000585          | SUCCESS    | OK
001593          | SUCCESS    | OK
004178          | SUCCESS    | OK
002011          | SUCCESS    |

In [6]:
import os

# Tạo thư mục chứa code
os.makedirs("src/featglac", exist_ok=True)

# Nội dung file code (Đã thêm PATCH FIX cho PyTorch 2.1.2)
inverter_code = r'''
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from typing import Optional, Union, Tuple, List

# --- 🔥 HOTFIX: VÁ LỖI COMPATIBILITY PYTORCH 2.1.2 ---
# Lỗi bạn gặp là do accelerate gọi hàm cũ 'grad_and_value' không còn trong torch 2.1
# Ta sẽ định nghĩa thủ công nó trỏ về 'torch.func.grad_and_value' (API mới)
try:
    import torch._functorch.eager_transforms
    if not hasattr(torch._functorch.eager_transforms, "grad_and_value"):
        print("🛠️ Đang áp dụng bản vá: torch._functorch.eager_transforms.grad_and_value")
        torch._functorch.eager_transforms.grad_and_value = torch.func.grad_and_value
except Exception as e:
    print(f"⚠️ Cảnh báo Patching: {e}")
# -----------------------------------------------------

class StableNullInverter:
    def __init__(self, pipe):
        self.pipe = pipe
        self.tokenizer = pipe.tokenizer
        self.text_encoder = pipe.text_encoder
        self.vae = pipe.vae
        self.unet = pipe.unet
        self.scheduler = pipe.scheduler
        self.device = pipe.device

    @torch.no_grad()
    def image2latent(self, image: Image.Image) -> torch.Tensor:
        """Chuyển ảnh PIL RGB -> Latent Tensor"""
        # Resize về 512x512
        image = image.resize((512, 512), resample=Image.Resampling.LANCZOS)
        
        # Normalize [-1, 1]
        img_np = np.array(image).astype(np.float32) / 127.5 - 1.0
        
        # To Tensor [B, C, H, W]
        img_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0)
        img_tensor = img_tensor.to(self.device).to(self.pipe.dtype)
        
        # Encode VAE
        latents = self.vae.encode(img_tensor).latent_dist.mean
        latents = latents * 0.18215
        return latents

    @torch.no_grad()
    def invert_input_image(
        self, 
        image: Image.Image, 
        prompt: str = "", 
        num_inference_steps: int = 50,
        guidance_scale: float = 1.0
    ) -> Tuple[torch.Tensor, torch.Tensor, List[torch.Tensor]]:
        """Thực hiện DDIM Inversion"""
        
        # 1. Encode ảnh gốc
        latents = self.image2latent(image)
        start_latents = latents.clone()
        
        # 2. Text Embeddings
        text_input = self.tokenizer(
            [prompt], 
            padding="max_length", 
            max_length=self.tokenizer.model_max_length, 
            truncation=True, 
            return_tensors="pt"
        )
        text_embeddings = self.text_encoder(text_input.input_ids.to(self.device))[0]

        # 3. Scheduler Setup
        self.scheduler.set_timesteps(num_inference_steps)
        reversed_timesteps = self.scheduler.timesteps
        if reversed_timesteps[0] > reversed_timesteps[-1]:
            reversed_timesteps = torch.flip(reversed_timesteps, [0])

        print(f"🔄 Đang Invert ({num_inference_steps} steps)...")
        
        # Lưu quỹ đạo (CPU để tiết kiệm VRAM)
        ddim_latents_all = [latents.cpu()] 

        # 4. Inversion Loop
        for i, t in enumerate(tqdm(reversed_timesteps)):
            noise_pred = self.unet(
                latents, 
                t, 
                encoder_hidden_states=text_embeddings
            ).sample

            alpha_prod_t = self.scheduler.alphas_cumprod[t]
            beta_prod_t = 1 - alpha_prod_t
            
            if i < len(reversed_timesteps) - 1:
                next_t = reversed_timesteps[i+1]
                alpha_prod_t_next = self.scheduler.alphas_cumprod[next_t]
            else:
                alpha_prod_t_next = self.scheduler.alphas_cumprod[0] 

            # DDIM Formula
            pred_original_sample = (latents - beta_prod_t ** 0.5 * noise_pred) / (alpha_prod_t ** 0.5)
            latents = alpha_prod_t_next ** 0.5 * pred_original_sample + (1 - alpha_prod_t_next) ** 0.5 * noise_pred
            
            ddim_latents_all.append(latents.cpu())

        print("✅ Inversion hoàn tất.")
        return start_latents, latents, ddim_latents_all
'''

with open("src/featglac/stable_null_inverter.py", "w") as f:
    f.write(inverter_code)
print("✅ Đã tạo file thư viện (Kèm bản vá lỗi PyTorch 2.1).")

✅ Đã tạo file thư viện (Kèm bản vá lỗi PyTorch 2.1).


In [7]:
# @title 🚑 EMERGENCY PATCH: Bỏ qua lỗi FSDP & Torch Distributed
import sys
from unittest.mock import MagicMock
import torch

print("🚑 ĐANG VÁ LỖI XUNG ĐỘT PYTORCH & ACCELERATE...")

# 1. Chặn import module gây lỗi trong Torch
# Lỗi '_fused_rms_norm_backward' nằm trong torch.distributed.tensor._ops
# Ta sẽ thay thế toàn bộ module này bằng Mock (đồ giả) để Python không chạy vào đó.
sys.modules["torch.distributed.tensor._ops"] = MagicMock()
sys.modules["torch.distributed.tensor"] = MagicMock()
sys.modules["torch.distributed.checkpoint"] = MagicMock()
sys.modules["torch.distributed.fsdp"] = MagicMock()

# 2. Chặn Accelerate gọi FSDP Utils
sys.modules["accelerate.utils.fsdp_utils"] = MagicMock()

# 3. Vá lại các lỗi cũ (nếu chưa chạy)
try:
    if not hasattr(torch.ops.aten, "_fused_rms_norm_backward"):
        # Tạo dummy attribute để nếu có module nào lọt lưới kiểm tra thì không bị crash
        setattr(torch.ops.aten, "_fused_rms_norm_backward", MagicMock())
except:
    pass

# 4. Patch Grad_and_value (cho Inversion)
try:
    import torch._functorch.eager_transforms
    if not hasattr(torch._functorch.eager_transforms, "grad_and_value"):
        torch._functorch.eager_transforms.grad_and_value = torch.func.grad_and_value
except: 
    pass

print("✅ ĐÃ VÁ XONG! BẠN CÓ THỂ CHẠY TIẾP CELL BÊN DƯỚI.")

🚑 ĐANG VÁ LỖI XUNG ĐỘT PYTORCH & ACCELERATE...
✅ ĐÃ VÁ XONG! BẠN CÓ THỂ CHẠY TIẾP CELL BÊN DƯỚI.


In [8]:
# @title 7️⃣ RUN MODULE 2 (FIXED PATH): Sửa lỗi thiếu StableNullInverter
import os
import sys
import gc
import glob
import time
import shutil
import types
import numpy as np
import torch
import pandas as pd
from PIL import Image
from unittest.mock import MagicMock
from pyspark.sql import SparkSession

# ================= CẤU HÌNH MODULE 2 =================
INPUT_DIR = "/kaggle/working/Depth-Aware-Editing/spark_output/module1"
OUTPUT_DIR = "/kaggle/working/Depth-Aware-Editing/spark_output/module2" 
NUM_INFERENCE_STEPS = 20
SPARK_MASTER = "local[1]" 

# ================= 🚑 EMERGENCY PATCH: FIX LỖI TORCH.MTIA =================
if not hasattr(torch, 'mtia'):
    torch.mtia = types.ModuleType('mtia')
if not hasattr(torch.mtia, '_set_stream_by_id'):
    torch.mtia._set_stream_by_id = lambda *args, **kwargs: None
if not hasattr(torch.mtia, 'set_stream'):
    torch.mtia.set_stream = lambda *args, **kwargs: None
if not hasattr(torch.mtia, 'synchronize'):
    torch.mtia.synchronize = lambda *args, **kwargs: None
if not hasattr(torch.mtia, 'device'):
    class FakeDevice: pass
    torch.mtia.device = FakeDevice

# ================= HÀM HỖ TRỢ DRIVER =================
def clean_memory():
    cleanup_vars = ['pipe', 'inverter', 'zoe_model', 'unet', 'vae']
    for var in cleanup_vars:
        if var in globals(): del globals()[var]
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()

# ================= LOGIC WORKER (CÓ FIX PATH) =================
def inversion_worker(iterator):
    import os
    import sys
    import torch
    import time
    import numpy as np
    from PIL import Image
    from diffusers import StableDiffusionPipeline 
    
    # --- 🔥 FIX QUAN TRỌNG: ÉP ĐƯỜNG DẪN CODE ---
    base_dir = "/kaggle/working/Depth-Aware-Editing"
    
    # Nạp cả thư mục gốc và thư mục src vào sys.path
    paths_to_inject = [
        base_dir,                       # Để import 'src.featglac...'
        os.path.join(base_dir, "src")   # Dự phòng
    ]
    
    for p in paths_to_inject:
        if os.path.exists(p) and p not in sys.path:
            sys.path.insert(0, p)
    # ---------------------------------------------

    try:
        # Import class Inverter từ code dự án
        from src.featglac.stable_null_inverter import StableNullInverter
    except ImportError as e:
        # Nếu lỗi, trả về thông báo debug chi tiết
        yield ("INIT_ERROR", f"Missing StableNullInverter: {str(e)}", 0, 0)
        return

    # 1. Load Model SD v1.5
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_id = "runwayml/stable-diffusion-v1-5"
    
    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            safety_checker=None
        ).to(device)
        
        # Khởi tạo Inverter
        inverter = StableNullInverter(pipe)
    except Exception as e:
        yield ("MODEL_LOAD_ERROR", str(e), 0, 0)
        return

    # 2. Loop xử lý ảnh
    for row in iterator:
        img_id = row.img_id
        img_path = row.img_path
        target_output_dir = row.output_dir 
        
        start_time = time.time()
        
        try:
            # Tạo thư mục output riêng cho ảnh này
            save_folder = os.path.join(target_output_dir, img_id)
            os.makedirs(save_folder, exist_ok=True)
            
            latent_path = os.path.join(save_folder, "inverted_latents.pt")
            
            # Check tồn tại để skip (Resume capability)
            if os.path.exists(latent_path):
                yield (img_id, "SKIPPED", 0, 0)
                continue

            # Load ảnh input từ Module 1
            input_image = Image.open(img_path).convert("RGB").resize((512, 512))
            
            # --- THỰC HIỆN INVERSION ---
            with torch.autocast("cuda"):
                # Hàm invert trả về (start_latents, inverted_latents, all_latents)
                # Ta cần lưu start_latents (z_T) để dùng cho Module 3
                _, inverted_latents, _ = inverter.invert_input_image(
                    input_image, 
                    prompt="", 
                    num_inference_steps=NUM_INFERENCE_STEPS
                )
            
            # Lưu file kết quả .pt
            torch.save(inverted_latents.cpu(), latent_path)
            
            # --- TÍNH TOÁN SAI SỐ (MSE) ---
            # Reconstruct ảnh để đo độ sai lệch
            with torch.no_grad():
                z0 = inverter.image2latent(input_image)
                recon_img_tensor = pipe.vae.decode(z0 / 0.18215).sample
                recon_img = (recon_img_tensor / 2 + 0.5).clamp(0, 1).cpu().permute(0, 2, 3, 1).numpy()[0]
                
            orig_img = np.array(input_image).astype(np.float32) / 255.0
            mse = np.mean((orig_img - recon_img) ** 2)
            
            process_time = time.time() - start_time
            yield (img_id, "SUCCESS", round(process_time, 2), round(mse, 6))
            
        except Exception as e:
            yield (img_id, f"ERROR: {str(e)}", 0, 0)

# ================= MAIN DRIVER =================
def main_module2():
    clean_memory()
    print(f"🚀 Khởi chạy Spark Module 2 (Inversion) - Output: {OUTPUT_DIR}")
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    spark = SparkSession.builder \
        .appName("Module2_Inversion_Fixed") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .master(SPARK_MASTER) \
        .getOrCreate()
    
    # 1. Quét file input từ Module 1 (File 'input.png' do Module 1 tạo ra)
    # Cấu trúc Module 1: spark_output/module1/{img_id}/input.png
    search_pattern = os.path.join(INPUT_DIR, "*", "input.png")
    input_files = glob.glob(search_pattern)
    
    if not input_files:
        print(f"❌ Không tìm thấy ảnh input nào tại: {INPUT_DIR}")
        print("👉 Hãy chắc chắn Module 1 đã chạy thành công và tạo file input.png")
        spark.stop()
        return
        
    print(f"📂 Tìm thấy {len(input_files)} ảnh đầu vào từ Module 1.")
    
    # Tạo DataFrame
    data = []
    for f in input_files:
        img_id = os.path.basename(os.path.dirname(f))
        data.append((img_id, f, OUTPUT_DIR))
    
    df = spark.createDataFrame(data, ["img_id", "img_path", "output_dir"])
    
    # 2. Chạy trên Spark
    df_part = df.repartition(1) 
    print("🔄 Đang thực hiện Inversion (Quá trình này tốn GPU)...")
    
    results_rdd = df_part.rdd.mapPartitions(inversion_worker)
    final_results = results_rdd.collect()
    
    # 3. Hiển thị kết quả
    print("\n✅ KẾT QUẢ MODULE 2:")
    print(f"{'IMAGE ID':<15} | {'STATUS':<15} | {'TIME (s)':<10} | {'MSE (Loss)'}")
    print("-" * 65)
    
    success_count = 0
    stats_data = []
    
    for img_id, status, p_time, mse in final_results:
        print(f"{img_id[:15]:<15} | {status[:15]:<15} | {p_time:<10} | {mse}")
        if status == "SUCCESS":
            success_count += 1
            stats_data.append({"Image ID": img_id, "Time": p_time, "MSE": mse})
            
    if stats_data:
        pd.DataFrame(stats_data).to_csv(os.path.join(OUTPUT_DIR, "module2_stats.csv"), index=False)
        
    print("-" * 65)
    print(f"🎉 Hoàn thành: {success_count}/{len(final_results)} ảnh.")
    
    spark.stop()
    clean_memory()

if __name__ == "__main__":
    main_module2()

🚀 Khởi chạy Spark Module 2 (Inversion) - Output: /kaggle/working/Depth-Aware-Editing/spark_output/module2
📂 Tìm thấy 50 ảnh đầu vào từ Module 1.
🔄 Đang thực hiện Inversion (Quá trình này tốn GPU)...


🛠️ Đang áp dụng bản vá: torch._functorch.eager_transforms.grad_and_value1) / 1]
/usr/local/lib/python3.12/dist-packages/diffusers/models/dual_transformer_2d.py:20: FutureWarning: `DualTransformer2DModel` is deprecated and will be removed in version 0.29. Importing `DualTransformer2DModel` from `diffusers.models.dual_transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.transformers.dual_transformer_2d import DualTransformer2DModel`, instead.
  deprecate("DualTransformer2DModel", "0.29", deprecation_message)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 6/6 [00:07<00:00,  1.27s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.


✅ KẾT QUẢ MODULE 2:
IMAGE ID        | STATUS          | TIME (s)   | MSE (Loss)
-----------------------------------------------------------------
005660          | SUCCESS         | 3.41       | 0.002601000014692545
003141          | SUCCESS         | 2.49       | 0.00446499977260828
004566          | SUCCESS         | 2.5        | 0.0048779998905956745
004710          | SUCCESS         | 2.5        | 0.00395999988541007
000419          | SUCCESS         | 2.5        | 0.002704000100493431
004656          | SUCCESS         | 2.5        | 0.00660800002515316
007284          | SUCCESS         | 2.51       | 0.001585999969393015
000603          | SUCCESS         | 2.52       | 0.0030869999900460243
003065          | SUCCESS         | 2.52       | 0.0021230001002550125
007141          | SUCCESS         | 2.54       | 0.002633000025525689
001669          | SUCCESS         | 2.53       | 0.006269000004976988
007355          | SUCCESS         | 2.53       | 0.0030759999062865973
004278      

In [9]:
# @title 8️⃣ RUN MODULE 3 (SPARK - FEATURE EXTRACTION): Trích xuất đặc trưng hàng loạt
import os
import sys
import gc
import glob
import time
import shutil
import types
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
from unittest.mock import MagicMock
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# ================= CẤU HÌNH MODULE 3 =================
# Input lấy từ output của Module 2
INPUT_DIR_M2 = "/kaggle/working/Depth-Aware-Editing/spark_output/module2"
OUTPUT_DIR_M3 = "/kaggle/working/Depth-Aware-Editing/spark_output/module3"
SPARK_MASTER = "local[1]"

# ================= 🚑 EMERGENCY PATCH: TORCH.MTIA =================
if not hasattr(torch, 'mtia'):
    torch.mtia = types.ModuleType('mtia')
if not hasattr(torch.mtia, '_set_stream_by_id'):
    torch.mtia._set_stream_by_id = lambda *args, **kwargs: None
if not hasattr(torch.mtia, 'set_stream'):
    torch.mtia.set_stream = lambda *args, **kwargs: None
if not hasattr(torch.mtia, 'synchronize'):
    torch.mtia.synchronize = lambda *args, **kwargs: None
if not hasattr(torch.mtia, 'device'):
    class FakeDevice: pass
    torch.mtia.device = FakeDevice

# ================= 🛡️ SYSTEM PATCH V4 (DIFFUSERS) =================
sys.modules["torch.onnx"] = MagicMock()
if "diffusers.hooks" not in sys.modules:
    hooks_pkg = types.ModuleType("diffusers.hooks")
    hooks_pkg.__path__ = [] 
    sys.modules["diffusers.hooks"] = hooks_pkg

if "diffusers.hooks.group_offloading" not in sys.modules:
    offload_mod = types.ModuleType("diffusers.hooks.group_offloading")
    sys.modules["diffusers.hooks.group_offloading"] = offload_mod
else:
    offload_mod = sys.modules["diffusers.hooks.group_offloading"]

sys.modules["diffusers.hooks"].group_offloading = offload_mod
def _fake_get_group_onload_device(*args, **kwargs): return torch.device("cpu")
def _fake_is_group_offload_enabled(*args, **kwargs): return False
offload_mod._get_group_onload_device = _fake_get_group_onload_device
offload_mod._is_group_offload_enabled = _fake_is_group_offload_enabled

try:
    import diffusers.models.model_loading_utils
    def _bypass_check(*args, **kwargs): return None
    diffusers.models.model_loading_utils.check_support_param_buffer_assignment = _bypass_check
except ImportError: pass

# ================= CLASS: FEATURE EXTRACTOR =================
class FeatureExtractor:
    def __init__(self, unet):
        self.unet = unet
        self.features = {}
        self.hooks = []
        
        # Hook vào các tầng Up-block (Decoder)
        for i, block in enumerate(unet.up_blocks):
            layer_name = f"up_block_{i}"
            hook = block.register_forward_hook(self._get_hook_fn(layer_name))
            self.hooks.append(hook)
            
        if hasattr(unet, "mid_block") and unet.mid_block is not None:
             hook = unet.mid_block.register_forward_hook(self._get_hook_fn("mid_block"))
             self.hooks.append(hook)

    def _get_hook_fn(self, layer_name):
        def hook(module, input, output):
            if isinstance(output, tuple): feature_map = output[0]
            else: feature_map = output
            self.features[layer_name] = feature_map.detach().cpu()
        return hook

    def remove_hooks(self):
        for hook in self.hooks: hook.remove()
        self.hooks = []

# ================= LOGIC WORKER (SPARK) =================
def feature_worker(iterator):
    import torch
    import os
    import numpy as np
    from diffusers import StableDiffusionPipeline, DDIMScheduler

    # Load Model (1 lần cho cả partition)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MODEL_ID = "runwayml/stable-diffusion-v1-5"
    
    try:
        scheduler = DDIMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
        pipe = StableDiffusionPipeline.from_pretrained(
            MODEL_ID, scheduler=scheduler, torch_dtype=torch.float32, # Feature extraction cần độ chính xác cao
            low_cpu_mem_usage=False, device_map=None, use_safetensors=True
        ).to(device)
        pipe.set_progress_bar_config(disable=True)
    except Exception as e:
        yield ("INIT_ERROR", str(e), "", 0, 0, 0, 0)
        return

    # Loop xử lý từng ảnh
    for row in iterator:
        img_id = row.img_id
        latent_path = row.latent_path
        
        try:
            # 1. Load Latent
            # File từ Module 2 là 'inverted_latents.pt'
            latents = torch.load(latent_path).to(device, dtype=torch.float32)

            # 2. Setup Extractor
            extractor = FeatureExtractor(pipe.unet)

            # 3. Forward Pass (Chạy qua UNet để kích hoạt features)
            t = torch.tensor([0]).to(device) # timestep 0
            with torch.no_grad():
                prompt_embeds, _ = pipe.encode_prompt(
                    prompt="", device=device, num_images_per_prompt=1, do_classifier_free_guidance=False
                )
                prompt_embeds = prompt_embeds.to(dtype=torch.float32)
                pipe.unet(latents, t, encoder_hidden_states=prompt_embeds)
            
            # 4. Lưu Features & Tính Stats
            save_dir = os.path.join(OUTPUT_DIR_M3, img_id)
            os.makedirs(save_dir, exist_ok=True)
            
            # Lưu stats của layer quan trọng nhất: 'up_block_2' (thường chứa semantic tốt nhất)
            target_layer = "up_block_2"
            stats_tuple = (0, 0, 0, 0)
            
            for name, tensor in extractor.features.items():
                # Lưu file .pt
                torch.save(tensor, os.path.join(save_dir, f"{name}.pt"))
                
                # Tính stats nếu là layer mục tiêu
                if name == target_layer:
                    arr = tensor.numpy()
                    stats_tuple = (
                        float(np.mean(arr)),
                        float(np.std(arr)),
                        float(np.min(arr)),
                        float(np.max(arr))
                    )
            
            # Vẽ heatmap đại diện (lưu ảnh png)
            if target_layer in extractor.features:
                feat = extractor.features[target_layer]
                heatmap = feat[0].mean(dim=0).float().numpy()
                plt.figure(figsize=(4, 4))
                plt.imshow(heatmap, cmap='magma')
                plt.axis('off')
                plt.tight_layout()
                plt.savefig(os.path.join(save_dir, "heatmap_viz.png"))
                plt.close()

            extractor.remove_hooks()
            
            yield (img_id, "SUCCESS", target_layer, *stats_tuple)
            
        except Exception as e:
            yield (img_id, f"ERROR: {str(e)}", "", 0, 0, 0, 0)

# ================= SPARK DRIVER =================
def main_module3():
    # Dọn dẹp
    if 'spark' in globals(): globals()['spark'].stop()
    gc.collect()
    torch.cuda.empty_cache()
    
    print(f"🚀 Khởi chạy Spark Module 3 (Feature Extraction)")
    print(f"📂 Input: {INPUT_DIR_M2}")
    print(f"📂 Output: {OUTPUT_DIR_M3}")
    
    os.makedirs(OUTPUT_DIR_M3, exist_ok=True)
    
    spark = SparkSession.builder \
        .appName("Module3_Features") \
        .config("spark.driver.memory", "4g") \
        .master(SPARK_MASTER) \
        .getOrCreate()

    # 1. Quét dữ liệu từ Module 2
    # Cấu trúc: module2/{img_id}/inverted_latents.pt
    search_pattern = os.path.join(INPUT_DIR_M2, "*", "inverted_latents.pt")
    latent_files = glob.glob(search_pattern)
    
    if not latent_files:
        print(f"❌ Không tìm thấy file 'inverted_latents.pt' nào trong {INPUT_DIR_M2}")
        spark.stop()
        return
    
    print(f"✅ Tìm thấy {len(latent_files)} bộ Latent để xử lý.")
    
    # 2. Tạo DataFrame
    data = []
    for f in latent_files:
        img_id = os.path.basename(os.path.dirname(f))
        data.append((img_id, f))
        
    df = spark.createDataFrame(data, ["img_id", "latent_path"])
    df_part = df.repartition(1)
    
    # 3. Chạy Worker
    print("🔄 Đang trích xuất đặc trưng (UNet Forward Pass)...")
    results_rdd = df_part.rdd.mapPartitions(feature_worker)
    
    # Định nghĩa Schema cho kết quả
    schema = StructType([
        StructField("Image_ID", StringType(), True),
        StructField("Status", StringType(), True),
        StructField("Layer", StringType(), True),
        StructField("Mean", FloatType(), True),
        StructField("Std", FloatType(), True),
        StructField("Min", FloatType(), True),
        StructField("Max", FloatType(), True)
    ])
    
    # Collect về DataFrame để hiển thị đẹp
    try:
        final_df = spark.createDataFrame(results_rdd, schema=schema)
        
        print("\n" + "="*80)
        print("📊 BẢNG THỐNG KÊ FEATURE MAPS (Lớp: up_block_2)")
        print("="*80)
        final_df.show(truncate=False)
        
        # Lưu bảng thống kê
        final_df.toPandas().to_csv(os.path.join(OUTPUT_DIR_M3, "feature_stats.csv"), index=False)
        
        # Đếm thành công
        success_count = final_df.filter("Status = 'SUCCESS'").count()
        print(f"🎉 Hoàn thành: {success_count}/{len(latent_files)} ảnh.")
        
    except Exception as e:
        print(f"❌ Lỗi khi hiển thị bảng: {e}")
        # In raw nếu lỗi DataFrame
        print(results_rdd.collect())

    spark.stop()

if __name__ == "__main__":
    main_module3()

🚀 Khởi chạy Spark Module 3 (Feature Extraction)
📂 Input: /kaggle/working/Depth-Aware-Editing/spark_output/module2
📂 Output: /kaggle/working/Depth-Aware-Editing/spark_output/module3
✅ Tìm thấy 50 bộ Latent để xử lý.
🔄 Đang trích xuất đặc trưng (UNet Forward Pass)...



📊 BẢNG THỐNG KÊ FEATURE MAPS (Lớp: up_block_2)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:09<00:00,  1.43s/it]


+--------+-------+----------+----------+---------+----------+---------+
|Image_ID|Status |Layer     |Mean      |Std      |Min       |Max      |
+--------+-------+----------+----------+---------+----------+---------+
|005660  |SUCCESS|up_block_2|0.27110583|2.880915 |-19.529297|27.833855|
|003141  |SUCCESS|up_block_2|0.2866447 |2.8925755|-19.278198|22.246082|
|004566  |SUCCESS|up_block_2|0.28827026|2.939748 |-18.113901|19.707815|
|004710  |SUCCESS|up_block_2|0.2710388 |2.927826 |-18.65343 |17.784754|
|000419  |SUCCESS|up_block_2|0.265091  |2.896575 |-20.215073|22.331396|
|004656  |SUCCESS|up_block_2|0.26360095|2.9126134|-17.390797|18.613302|
|007284  |SUCCESS|up_block_2|0.28630087|2.8744402|-17.68321 |26.28304 |
|000603  |SUCCESS|up_block_2|0.2912592 |2.93584  |-18.605118|19.507841|
|003065  |SUCCESS|up_block_2|0.2803051 |2.8965063|-17.611792|21.625946|
|007141  |SUCCESS|up_block_2|0.24045257|2.885831 |-17.944359|17.95273 |
|001669  |SUCCESS|up_block_2|0.26341736|2.929938 |-17.766838|19.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:09<00:00,  1.36s/it]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:09<00:00,  1.37s/it] 1]


🎉 Hoàn thành: 50/50 ảnh.


In [ ]:
# ==============================================================================
# CELL "CẤP CỨU": GIẢI PHÓNG RAM CPU VÀ GPU TRIỆT ĐỂ
# ==============================================================================
import torch
import gc
import ctypes
import sys

print("🧹 Đang tiến hành tổng vệ sinh bộ nhớ (CPU & GPU)...")

# 1. Xóa các biến lớn trong Python
# Danh sách các tên biến thường dùng trong notebook này
targets = ['model', 'diff_handles', 'activations', 'loaded_feats', 
           'init_noise', 'depth_tensor', 'mpi_masks', 'results', 
           'sd', 'state_dict', 'checkpoint']

for t in targets:
    if t in globals():
        del globals()[t]
        print(f"   - Đã xóa biến Python: {t}")

# 2. Ép Garbage Collector chạy
gc.collect()

# 3. QUAN TRỌNG: Ép Linux thu hồi bộ nhớ RAM (Malloc Trim)
# Lệnh này giúp RAM giảm ngay lập tức trên Kaggle/Colab
try:
    libc = ctypes.CDLL("libc.so.6")
    libc.malloc_trim(0)
    print("   - Đã chạy lệnh hệ thống: libc.malloc_trim(0)")
except Exception as e:
    print(f"   - Không gọi được libc (Không sao): {e}")

# 4. Xả Cache GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("   - Đã xả CUDA Cache.")

print("-" * 30)
print("✅ HOÀN TẤT! HÃY KIỂM TRA LẠI THANH RAM Ở GÓC TRÊN.")
print("👉 Sau đó chạy lại Cell Load Model bên dưới.")

In [ ]:
# ==============================================================================
# BƯỚC 2: LOAD MODEL SD 1.5 INPAINTING (BẢN VÁ V11 - FIX INPAINTING CONFIG)
# ==============================================================================
import torch
import os
import sys
import requests
import glob
import gc
import ctypes
import types
import yaml
from unittest.mock import MagicMock

print("🚑 ĐANG KÍCH HOẠT HỆ THỐNG VÁ LỖI V11 (CHANNEL MISMATCH FIX)...")

# --- 1. SỬA FILE CONFIG TỪ 4 KÊNH -> 9 KÊNH (LỖI CỦA BẠN) ---
# Tải config chuẩn trước
config_path = "cldm_v15_fixed.yaml"
if not os.path.exists(config_path):
    url = "https://raw.githubusercontent.com/lllyasviel/ControlNet/main/models/cldm_v15.yaml"
    try:
        with open(config_path, "wb") as f: f.write(requests.get(url).content)
    except: pass

# Đọc và sửa config
try:
    with open(config_path, 'r') as f:
        config_data = yaml.safe_load(f)
    
    # Kiểm tra và sửa in_channels
    current_channels = config_data['model']['params']['unet_config']['params']['in_channels']
    print(f"   ℹ️ Config gốc có {current_channels} kênh đầu vào.")
    
    if current_channels != 9:
        print("   🔧 Đang chuyển đổi Config sang chế độ Inpainting (9 kênh)...")
        config_data['model']['params']['unet_config']['params']['in_channels'] = 9
        
        # Lưu ra file mới
        new_config_path = "cldm_v15_inpainting_patched.yaml"
        with open(new_config_path, 'w') as f:
            yaml.dump(config_data, f)
        config_path = new_config_path # Trỏ sang file mới
        print(f"   ✅ Đã tạo config mới: {config_path}")
    else:
        print("   ✅ Config đã chuẩn 9 kênh.")

except Exception as e:
    print(f"   ⚠️ Lỗi khi sửa config: {e}")

# --- 2. FIX CÁC LỖI KHÁC (GIỮ NGUYÊN TỪ V10) ---
# Fix missing anydoor config
required_config_path = "/kaggle/working/configs/anydoor.yaml"
if not os.path.exists(required_config_path):
    os.makedirs(os.path.dirname(required_config_path), exist_ok=True)
    dummy_content = """
model:
  target: ldm.models.diffusion.ddpm.LatentDiffusion
  params:
    linear_start: 0.00085
    linear_end: 0.0120
    num_timesteps_cond: 1
    log_every_t: 200
    timesteps: 1000
    first_stage_key: "jpg"
    cond_stage_key: "txt"
    image_size: 64
    channels: 4
    cond_stage_trainable: false
    conditioning_key: crossattn
    monitor: val/loss_simple_ema
    scale_factor: 0.18215
    use_ema: False
    cond_stage_config:
      target: ldm.modules.encoders.modules.FrozenOpenCLIPEmbedder
      params:
        freeze: True
        layer: "penultimate"
      weight: "weights/dinov2_vitg14_pretrain.pth"
"""
    with open(required_config_path, "w") as f: f.write(dummy_content)

# Patch ONNX & FSDP
fake_onnx = types.ModuleType("torch.onnx")
fake_onnx.__path__ = []
fake_onnx.symbolic_helper = MagicMock()
fake_onnx.utils = MagicMock()
fake_onnx.symbolic_opset11 = MagicMock()
fake_onnx.symbolic_opset9 = MagicMock()
fake_onnx.symbolic_opset10 = MagicMock()
fake_onnx.register_custom_op_symbolic = MagicMock(return_value=None)
fake_onnx.unregister_custom_op_symbolic = MagicMock(return_value=None)
fake_onnx.select_model_mode_for_export = MagicMock(return_value=None)

sys.modules["torch.onnx"] = fake_onnx
sys.modules["torch.onnx.symbolic_helper"] = fake_onnx.symbolic_helper
sys.modules["torch.onnx.utils"] = fake_onnx.utils
sys.modules["torch.onnx.symbolic_opset11"] = fake_onnx.symbolic_opset11
fake_onnx.symbolic_helper.parse_args = MagicMock(return_value=lambda x: x)
fake_onnx.symbolic_helper._parse_arg = MagicMock(return_value=lambda x: x)

class MockClass:
    def __init__(self, *args, **kwargs): pass

fake_fsdp = types.ModuleType("torch.distributed.fsdp")
fake_fsdp.__path__ = []
fake_fsdp.CPUOffload = MockClass
fake_fsdp.FullyShardedDataParallel = MockClass
fake_fsdp.MixedPrecision = MockClass
fake_fsdp.BackwardPrefetch = MockClass
fake_fsdp.ShardingStrategy = MockClass
fake_fsdp.StateDictType = MockClass
fake_fsdp.FullStateDictConfig = MockClass

sys.modules["torch.distributed.fsdp"] = fake_fsdp
sys.modules["torch.distributed.fsdp.fully_sharded_data_parallel"] = fake_fsdp
sys.modules["torch.distributed.fsdp.sharded_grad_scaler"] = MagicMock()
sys.modules["torch.distributed.fsdp.wrap"] = MagicMock()

fake_algo = types.ModuleType("torch.distributed.algorithms")
sys.modules["torch.distributed.algorithms"] = fake_algo
sys.modules["torch.distributed.algorithms.model_averaging"] = MagicMock()
sys.modules["torch.distributed.algorithms.model_averaging.averagers"] = MagicMock()

print("   ✅ [2/2] Đã vá Environment & Config.")
print("🚀 TIẾP TỤC LOAD MODEL...")
# -----------------------------------------------------------

# Setup path
sys.path.append(os.getcwd())
if os.path.exists("/kaggle/working/Depth-Aware-Editing"):
    sys.path.append("/kaggle/working/Depth-Aware-Editing")

try:
    from cldm.model import create_model, load_state_dict
    from cldm.ddim_hacked_mpi_featguidance import DDIMSampler
except ImportError:
    if os.path.exists("/kaggle/temp/Depth-Aware-Editing"):
        os.chdir("/kaggle/temp/Depth-Aware-Editing")
        sys.path.append("/kaggle/temp/Depth-Aware-Editing")
        from cldm.model import create_model, load_state_dict
        from cldm.ddim_hacked_mpi_featguidance import DDIMSampler
    else:
        print("❌ Không tìm thấy thư mục source code!")

# Chọn GPU 0
target_device = "cuda:0"
ckpt_path = '/kaggle/working/Depth-Aware-Editing/weights/sd-v1-5-inpainting.ckpt'

# Sử dụng config_path đã được vá ở trên
print(f"ℹ️ Sử dụng Config: {config_path}")

try:
    print("📦 Tạo khung Model (CPU)...")
    import warnings
    warnings.filterwarnings("ignore")
    
    model = create_model(config_path).cpu()
    
    print("📂 Đang đọc checkpoint...")
    if not os.path.exists(ckpt_path):
        found = glob.glob("/kaggle/working/**/sd-v1-5-inpainting.ckpt", recursive=True)
        if found: ckpt_path = found[0]
        else: 
            print("⬇️ Đang tải weights dự phòng...")
            os.system(f"wget -q -O {ckpt_path} https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt")
            
    if not os.path.exists(ckpt_path): raise FileNotFoundError("Không tìm thấy file checkpoint!")

    # Load Weights
    sd = load_state_dict(ckpt_path, location='cpu')
    if 'state_dict' in sd: sd = sd['state_dict']
    
    print("⏳ Đang nạp weights vào model (Strict=False)...")
    m, u = model.load_state_dict(sd, strict=False)
    print(f"   -> Model loaded. Missing keys: {len(m)}, Unexpected keys: {len(u)}")
    
    # Clean RAM
    del sd
    gc.collect()
    try: ctypes.CDLL("libc.so.6").malloc_trim(0) 
    except: pass

    # Move to GPU (FIX: Dùng FP32 thay vì FP16)
    print(f"🚚 Đang đẩy model sang {target_device} (FP32)...")
    model = model.float().to(target_device)  # FIX: float() thay vì half()
    
    diff_handles = DDIMSampler(model)
    device = torch.device(target_device)
    
    print("="*60)
    print("🎉 LOAD MODEL THÀNH CÔNG!")
    print(f"   - Checkpoint: {os.path.basename(ckpt_path)}")
    print(f"   - VRAM Used: {torch.cuda.memory_allocated(device)/1e9:.2f} GB")
    print("👉 Model đã sẵn sàng. Hãy chạy tiếp Cell Module 4.")
    print("="*60)

except Exception as e:
    print(f"❌ Vẫn còn lỗi: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# @title 🚀 MODULE 4: Scene Composition (Simplified + Spark)
import os
import sys
import gc
import torch
import numpy as np
from PIL import Image
import torch.nn. functional as F
from pyspark.sql import SparkSession
from pyspark.sql. types import StructType, StructField, StringType
import matplotlib. pyplot as plt

print("🚀 MODULE 4: SIMPLIFIED SCENE COMPOSITION")

# ================= CONFIG =================
BASE_WORK_DIR = "/kaggle/working/Depth-Aware-Editing/spark_output"
DIR_M1 = os.path.join(BASE_WORK_DIR, "module1")
DIR_M2 = os.path.join(BASE_WORK_DIR, "module2")
DIR_M3 = os.path.join(BASE_WORK_DIR, "module3")
OUTPUT_DIR_M4 = os.path.join(BASE_WORK_DIR, "module4")
PROMPT = "a high quality photo of a living room"
SPARK_MASTER = "local[1]"

def clean_memory():
    for var in ['pipe', 'model']: 
        if var in globals(): del globals()[var]
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()

clean_memory()

# ================= WORKER =================
def generation_worker(iterator):
    import os, sys, torch, numpy as np
    from PIL import Image
    from scipy.ndimage import gaussian_filter
    
    sys.path.insert(0, "/kaggle/working/Depth-Aware-Editing")
    
    try:
        from diffusers import StableDiffusionImg2ImgPipeline
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            safety_checker=None
        ).to(device)
        pipe.set_progress_bar_config(disable=True)
    except Exception as e:
        yield ("INIT_ERROR", str(e), "")
        return
    
    for row in iterator:
        img_id = row. img_id
        try:
            path_m1 = os.path.join(DIR_M1, img_id)
            
            # Load image & mask
            input_img = Image.open(os.path.join(path_m1, "input.png")).convert("RGB").resize((512, 512))
            mask_pil = Image.open(os. path.join(path_m1, "mask.png")).convert("L").resize((512, 512))
            mask_np = np.array(mask_pil) / 255.0
            
            # Blur masked region
            input_np = np.array(input_img)
            blurred = gaussian_filter(input_np, sigma=[5, 5, 0])
            mask_3ch = np.stack([mask_np]*3, axis=-1)
            blended = input_np * (1 - mask_3ch) + blurred * mask_3ch
            blended_img = Image.fromarray(blended.astype(np.uint8))
            
            # Generate
            with torch.no_grad():
                output = pipe(
                    prompt=PROMPT,
                    image=blended_img,
                    strength=0.75,
                    guidance_scale=7.5,
                    num_inference_steps=50
                ).images[0]
            
            # Save
            save_dir = os.path.join(OUTPUT_DIR_M4, img_id)
            os.makedirs(save_dir, exist_ok=True)
            output_path = os.path.join(save_dir, "final_result.png")
            output. save(output_path)
            
            # Save comparison
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            axes[0].imshow(input_img); axes[0].set_title("Input"); axes[0].axis('off')
            axes[1].imshow(mask_pil, cmap='gray'); axes[1].set_title("Mask"); axes[1].axis('off')
            axes[2].imshow(output); axes[2].set_title("Output"); axes[2].axis('off')
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "comparison.png"), dpi=100)
            plt.close()
            
            yield (img_id, "SUCCESS", output_path)
            
        except Exception as e: 
            yield (img_id, "ERROR", str(e)[:80])

# ================= MAIN =================
def main():
    if not os.path.exists(DIR_M1):
        print("❌ Module 1 chưa chạy!")
        return
    
    img_ids = [d for d in os.listdir(DIR_M1) if os.path.isdir(os.path.join(DIR_M1, d))]
    if not img_ids:
        print("❌ Không có dữ liệu")
        return
    
    print(f" Found {len(img_ids)} images")
    os.makedirs(OUTPUT_DIR_M4, exist_ok=True)
    
    # Spark
    spark = SparkSession.builder \
        .appName("Module4") \
        .config("spark. driver.memory", "4g") \
        .master(SPARK_MASTER) \
        .getOrCreate()
    
    spark.sparkContext. setLogLevel("ERROR")
    
    schema = StructType([StructField("img_id", StringType(), True)])
    df = spark.createDataFrame([(i,) for i in img_ids], schema=schema)
    
    print("\n Processing...")
    results = df.rdd.mapPartitions(generation_worker).collect()
    
    # Results
    print("\n" + "="*80)
    print(f"{'IMAGE ID':<20} | {'STATUS':<15} | {'OUTPUT'}")
    print("-"*80)
    
    success = 0
    for img_id, status, note in results:
        display_note = note[-40: ] if len(note) > 40 else note
        if status == "SUCCESS":
            success += 1
            print(f"{img_id:<20} | {status:<13} | ... {display_note}")
        else:
            print(f"{img_id:<20} | {status:<13} | {display_note}")
    
    print("-"*80)
    print(f"Success: {success}/{len(results)}")
    print(f"Output: {OUTPUT_DIR_M4}")
    print("="*80)
    
    spark.stop()

if __name__ == "__main__":
    main()